In [89]:
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.model_selection import cross_val_score, cross_validate
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from collections import Counter
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, FunctionTransformer


<h2>XGBoost Model<h2>

In [90]:
df_num = pd.read_csv('./df_num.csv')

In [91]:
columns_to_drop = [
    'StreamingMovies_No internet service',
    'StreamingTV_No internet service',
    'TechSupport_No internet service',
    'DeviceProtection_No internet service',
    'OnlineBackup_No internet service',
    'OnlineSecurity_No internet service',
    'MultipleLines_No phone service'
]

# Eliminar las columnas del DataFrame
df_num = df_num.drop(columns=columns_to_drop)

# Verificar que las columnas han sido eliminadas
df_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 25 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   SeniorCitizen                          7043 non-null   int64  
 1   Partner                                7043 non-null   int64  
 2   Dependents                             7043 non-null   int64  
 3   tenure                                 7043 non-null   int64  
 4   PhoneService                           7043 non-null   int64  
 5   PaperlessBilling                       7043 non-null   int64  
 6   MonthlyCharges                         7043 non-null   float64
 7   TotalCharges                           7043 non-null   float64
 8   Churn                                  7043 non-null   int64  
 9   gender_Male                            7043 non-null   int64  
 10  MultipleLines_Yes                      7043 non-null   int64  
 11  Inte

### Model 1

In [92]:
X = df_num.drop(columns=['Churn'])
y = df_num['Churn']

In [93]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [94]:
model1 = XGBClassifier(
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss' 
)

In [95]:
model1.fit(X_train, y_train)

/Users/laraaldalur/Desktop/thebridge/2409_dsft_lara_core/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:27:43] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [96]:
y_test_pred = model1.predict(X_test)

cm = confusion_matrix(y_test, y_test_pred)
print("Confusion Matrix:")
print(cm)
print(classification_report(y_test, y_test_pred))

#Verify the overfitting
y_train_pred = model1.predict(X_train)
train_score = accuracy_score(y_train, y_train_pred)
y_test_pred = model1.predict(X_test)
test_score = accuracy_score(y_test, y_test_pred)
difference = train_score - test_score

print(f"Train Score: {train_score:.4f}")
print(f"Test Score: {test_score:.4f}")
print(f"Difference (Train - Test): {difference:.4f}")

roc_auc = roc_auc_score(y_test, y_test_pred)
print(f"ROC-AUC: {roc_auc:.4f}")

Confusion Matrix:
[[904 131]
 [177 197]]
              precision    recall  f1-score   support

           0       0.84      0.87      0.85      1035
           1       0.60      0.53      0.56       374

    accuracy                           0.78      1409
   macro avg       0.72      0.70      0.71      1409
weighted avg       0.77      0.78      0.78      1409

Train Score: 0.9340
Test Score: 0.7814
Difference (Train - Test): 0.1526
ROC-AUC: 0.7001


### Model 2

We will use a RandomSearch to analyze the best hyperparameters and try to improve the model.

In [97]:
# Hiperparámetros para buscar
param_distributions = {
    'n_estimators': [50, 100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 10],
    'min_child_weight': [1, 3, 5, 7],
    'subsample': [0.5, 0.7, 0.9, 1.0],
    'colsample_bytree': [0.5, 0.7, 0.9, 1.0],
    'gamma': [0, 0.1, 0.2, 0.3]
}

In [98]:
# Crear modelo base
model2 = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')

In [99]:
# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=model2,
    param_distributions=param_distributions,
    n_iter=30,  # Número de combinaciones aleatorias
    scoring='f1', 
    cv=5,  # Validación cruzada
    random_state=42,
    verbose=2,
    n_jobs=-1  # Paralelización
)

In [100]:
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


/Users/laraaldalur/Desktop/thebridge/2409_dsft_lara_core/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:27:45] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/laraaldalur/Desktop/thebridge/2409_dsft_lara_core/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:27:45] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/laraaldalur/Desktop/thebridge/2409_dsft_lara_core/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:27:45] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/laraaldalur/Desktop/thebridge/2409_dsft_lara_core/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:27:45] WAR

[CV] END colsample_bytree=0.7, gamma=0.3, learning_rate=0.01, max_depth=5, min_child_weight=5, n_estimators=100, subsample=0.9; total time=   0.1s
[CV] END colsample_bytree=1.0, gamma=0.3, learning_rate=0.05, max_depth=7, min_child_weight=7, n_estimators=50, subsample=1.0; total time=   0.1s
[CV] END colsample_bytree=1.0, gamma=0.3, learning_rate=0.05, max_depth=7, min_child_weight=7, n_estimators=50, subsample=1.0; total time=   0.1s
[CV] END colsample_bytree=0.7, gamma=0.3, learning_rate=0.01, max_depth=5, min_child_weight=5, n_estimators=100, subsample=0.9; total time=   0.1s
[CV] END colsample_bytree=0.7, gamma=0.3, learning_rate=0.01, max_depth=5, min_child_weight=5, n_estimators=100, subsample=0.9; total time=   0.1s
[CV] END colsample_bytree=0.7, gamma=0.3, learning_rate=0.01, max_depth=5, min_child_weight=5, n_estimators=100, subsample=0.9; total time=   0.1s
[CV] END colsample_bytree=0.7, gamma=0.3, learning_rate=0.01, max_depth=5, min_child_weight=5, n_estimators=100, subsamp

/Users/laraaldalur/Desktop/thebridge/2409_dsft_lara_core/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:27:45] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/laraaldalur/Desktop/thebridge/2409_dsft_lara_core/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:27:45] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/laraaldalur/Desktop/thebridge/2409_dsft_lara_core/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:27:45] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/laraaldalur/Desktop/thebridge/2409_dsft_lara_core/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:27:45] WAR

[CV] END colsample_bytree=0.5, gamma=0, learning_rate=0.2, max_depth=5, min_child_weight=3, n_estimators=300, subsample=0.5; total time=   0.2s
[CV] END colsample_bytree=0.5, gamma=0, learning_rate=0.2, max_depth=5, min_child_weight=3, n_estimators=300, subsample=0.5; total time=   0.2s
[CV] END colsample_bytree=0.7, gamma=0.1, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=300, subsample=0.9; total time=   0.1s
[CV] END colsample_bytree=0.5, gamma=0, learning_rate=0.2, max_depth=5, min_child_weight=3, n_estimators=300, subsample=0.5; total time=   0.1s
[CV] END colsample_bytree=0.7, gamma=0.1, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=300, subsample=0.9; total time=   0.1s
[CV] END colsample_bytree=0.7, gamma=0.1, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=300, subsample=0.9; total time=   0.1s
[CV] END colsample_bytree=0.5, gamma=0, learning_rate=0.2, max_depth=5, min_child_weight=3, n_estimators=300, subsample=0.5; to

...

[CV] END colsample_bytree=1.0, gamma=0.1, learning_rate=0.05, max_depth=10, min_child_weight=5, n_estimators=200, subsample=1.0; total time=   0.2s
[CV] END colsample_bytree=0.7, gamma=0.2, learning_rate=0.2, max_depth=3, min_child_weight=5, n_estimators=100, subsample=0.7; total time=   0.1s
[CV] END colsample_bytree=0.7, gamma=0.2, learning_rate=0.2, max_depth=3, min_child_weight=5, n_estimators=100, subsample=0.7; total time=   0.1s
[CV] END colsample_bytree=1.0, gamma=0.1, learning_rate=0.05, max_depth=10, min_child_weight=5, n_estimators=200, subsample=1.0; total time=   0.2s
[CV] END colsample_bytree=0.7, gamma=0.2, learning_rate=0.2, max_depth=3, min_child_weight=5, n_estimators=100, subsample=0.7; total time=   0.1s
[CV] END colsample_bytree=1.0, gamma=0.1, learning_rate=0.05, max_depth=10, min_child_weight=5, n_estimators=200, subsample=1.0; total time=   0.2s
[CV] END colsample_bytree=0.7, gamma=0.2, learning_rate=0.2, max_depth=3, min_child_weight=5, n_estimators=100, subsam

/Users/laraaldalur/Desktop/thebridge/2409_dsft_lara_core/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:27:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/laraaldalur/Desktop/thebridge/2409_dsft_lara_core/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:27:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/laraaldalur/Desktop/thebridge/2409_dsft_lara_core/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:27:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/laraaldalur/Desktop/thebridge/2409_dsft_lara_core/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:27:47] WAR

[CV] END colsample_bytree=1.0, gamma=0.3, learning_rate=0.05, max_depth=7, min_child_weight=3, n_estimators=300, subsample=0.7; total time=   0.2s
[CV] END colsample_bytree=1.0, gamma=0.3, learning_rate=0.05, max_depth=7, min_child_weight=3, n_estimators=300, subsample=0.7; total time=   0.2s
[CV] END colsample_bytree=1.0, gamma=0.3, learning_rate=0.05, max_depth=7, min_child_weight=3, n_estimators=300, subsample=0.7; total time=   0.2s
[CV] END colsample_bytree=0.7, gamma=0.1, learning_rate=0.01, max_depth=7, min_child_weight=7, n_estimators=300, subsample=1.0; total time=   0.2s
[CV] END colsample_bytree=0.7, gamma=0.1, learning_rate=0.01, max_depth=7, min_child_weight=7, n_estimators=300, subsample=1.0; total time=   0.2s
[CV] END colsample_bytree=0.7, gamma=0.1, learning_rate=0.01, max_depth=7, min_child_weight=7, n_estimators=300, subsample=1.0; total time=   0.2s
[CV] END colsample_bytree=0.7, gamma=0, learning_rate=0.2, max_depth=7, min_child_weight=7, n_estimators=200, subsampl

/Users/laraaldalur/Desktop/thebridge/2409_dsft_lara_core/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:27:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None, gamma=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning...
                                           n_estimators=None, n_jobs=None,
                                           num_parallel_tree=None,
                                           random_state=42, ...),
                   n_iter=30, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.5, 0.7, 0.9,
                                                             1.0],
                                        'gamma': [0, 0.1, 0.2, 0.3],
                                        'learning_rate': [0.01, 0.05, 0.1, 0.2],
                                        'max_depth': [3, 5, 7, 10],
                                        'min_child_weight': [1, 3, 5, 7],
                                        'n_estimators': [50, 100, 200, 300],
                                        'subsample': [0.5, 0.7, 0.9, 1.0]},
                   random_state=42, scoring='f1', verbose=2)

In [101]:
print("Mejores parámetros:", random_search.best_params_)
print("Mejor ROC-AUC en validación cruzada:", random_search.best_score_)

best_xgb_model = random_search.best_estimator_

Mejores parámetros: {'subsample': 0.5, 'n_estimators': 100, 'min_child_weight': 7, 'max_depth': 3, 'learning_rate': 0.05, 'gamma': 0.3, 'colsample_bytree': 1.0}
Mejor ROC-AUC en validación cruzada: 0.5926759211950285


In [102]:
y_test_pred = best_xgb_model.predict(X_test)

cm = confusion_matrix(y_test, y_test_pred)
print("Confusion Matrix:")
print(cm)
print(classification_report(y_test, y_test_pred))

#Verify the overfitting
y_train_pred = best_xgb_model.predict(X_train)
train_score = accuracy_score(y_train, y_train_pred)
y_test_pred = best_xgb_model.predict(X_test)
test_score = accuracy_score(y_test, y_test_pred)
difference = train_score - test_score

print(f"Train Score: {train_score:.4f}")
print(f"Test Score: {test_score:.4f}")
print(f"Difference (Train - Test): {difference:.4f}")

roc_auc = roc_auc_score(y_test, y_test_pred)
print(f"ROC-AUC: {roc_auc:.4f}")


Confusion Matrix:
[[937  98]
 [178 196]]
              precision    recall  f1-score   support

           0       0.84      0.91      0.87      1035
           1       0.67      0.52      0.59       374

    accuracy                           0.80      1409
   macro avg       0.75      0.71      0.73      1409
weighted avg       0.79      0.80      0.80      1409

Train Score: 0.8166
Test Score: 0.8041
Difference (Train - Test): 0.0125
ROC-AUC: 0.7147


### Model 3

Main Model: XGBoost with Adjusted Hyperparameters
<br><br>
XGBoost is the most robust model for capturing complex patterns. However, to improve the balance between recall and precision:
<br><br>
Increase max_depth from 3 to 4 or 5 to capture deeper relationships.
Adjust learning_rate to 0.01 and increase n_estimators to 300-500 for fine tuning.
Experiment with subsample=0.7-0.8 to use more data in each iteration.
Optimise the decision threshold to prioritise recall of the churn class without compromising accuracy too much.

In [128]:
model3 = XGBClassifier(
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss',
    subsample=0.8,
    n_estimators=400,
    min_child_weight=7,
    max_depth=6,
    learning_rate=0.005,
    gamma=0.4,
    colsample_bytree=1.0
)

# Entrenar el modelo
model3.fit(X_train, y_train)

/Users/laraaldalur/Desktop/thebridge/2409_dsft_lara_core/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [13:08:38] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=1.0, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=0.4, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.005, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=7, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=400,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [129]:
y_test_pred = model3.predict(X_test)

cm = confusion_matrix(y_test, y_test_pred)
print("Confusion Matrix:")
print(cm)
print(classification_report(y_test, y_test_pred))

#Verify the overfitting
y_train_pred = model3.predict(X_train)
train_score = accuracy_score(y_train, y_train_pred)
y_test_pred = model3.predict(X_test)
test_score = accuracy_score(y_test, y_test_pred)
difference = train_score - test_score

print(f"Train Score: {train_score:.4f}")
print(f"Test Score: {test_score:.4f}")
print(f"Difference (Train - Test): {difference:.4f}")

roc_auc = roc_auc_score(y_test, y_test_pred)
print(f"ROC-AUC: {roc_auc:.4f}")

Confusion Matrix:
[[954  81]
 [195 179]]
              precision    recall  f1-score   support

           0       0.83      0.92      0.87      1035
           1       0.69      0.48      0.56       374

    accuracy                           0.80      1409
   macro avg       0.76      0.70      0.72      1409
weighted avg       0.79      0.80      0.79      1409

Train Score: 0.8211
Test Score: 0.8041
Difference (Train - Test): 0.0170
ROC-AUC: 0.7002


### Voting Classifier 

Combining XGBoost and logistic regression can improve recall while maintaining accuracy. Logistic regression will provide interpretability and good capacity for linear data. XGBoost will provide flexibility in capturing non-linear relationships.

In [105]:
# We scale and perform a logarithmic transformation for Regression model.

In [106]:
X_train_logistic = X_train.copy()
X_test_logistic = X_test.copy()

In [107]:
X_train_logistic['Log_TotalCharges'] = np.log1p(X_train['TotalCharges'])
X_train_logistic['Log_MonthlyCharges'] = np.log1p(X_train['MonthlyCharges'])
X_test_logistic['Log_TotalCharges'] = np.log1p(X_test['TotalCharges'])
X_test_logistic['Log_MonthlyCharges'] = np.log1p(X_test['MonthlyCharges'])

In [108]:
X_train_logistic = X_train_logistic.drop(columns=['TotalCharges', 'MonthlyCharges'])
X_test_logistic = X_test_logistic.drop(columns=['TotalCharges', 'MonthlyCharges'])

In [109]:
scaler = StandardScaler()
X_train_logistic_scaled = scaler.fit_transform(X_train_logistic)
X_test_logistic_scaled = scaler.transform(X_test_logistic)

In [110]:
# Modelo de Regresión Logística
log_model = LogisticRegression(
    C=0.5908361216819946,  # Hiperparámetros ajustados previamente
    class_weight='balanced',
    max_iter=100,
    penalty='l2',
    solver='newton-cg',
    random_state=42
)

In [111]:
# Modelo XGBoost
xgb_model = XGBClassifier(
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss',
    subsample=0.8,
    n_estimators=400,
    min_child_weight=7,
    max_depth=5,
    learning_rate=0.01,
    gamma=0.3,
    colsample_bytree=1.0
)

In [112]:
# Ensamble con VotingClassifier
ensemble_model = VotingClassifier(
    estimators=[('logistic', log_model), ('xgboost', xgb_model)],
    voting='soft' 
)

ensemble_model.fit(X_train_logistic_scaled, y_train)

/Users/laraaldalur/Desktop/thebridge/2409_dsft_lara_core/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:27:48] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


VotingClassifier(estimators=[('logistic',
                              LogisticRegression(C=0.5908361216819946,
                                                 class_weight='balanced',
                                                 random_state=42,
                                                 solver='newton-cg')),
                             ('xgboost',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=1.0, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metr...
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.01, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=5,
                                            max_leaves=None, min_child_weight=7,
                                            missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=400, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=42, ...))],
                 voting='soft')

In [113]:
# Predicciones del ensamble
y_pred_proba_ensemble = ensemble_model.predict_proba(X_test_logistic_scaled)[:, 1]
y_pred_ensemble = (y_pred_proba_ensemble >= 0.5).astype(int)

conf_matrix_ensemble = confusion_matrix(y_test, y_pred_ensemble)
class_report_ensemble = classification_report(y_test, y_pred_ensemble)
roc_auc_ensemble = roc_auc_score(y_test, y_pred_proba_ensemble)

#Verify the overfitting
y_train_pred = ensemble_model.predict(X_train_logistic_scaled)
train_score = accuracy_score(y_train, y_train_pred)
y_test_pred = ensemble_model.predict(X_test_logistic_scaled)
test_score = accuracy_score(y_test, y_test_pred)
difference = train_score - test_score

print(f"Train Score: {train_score:.4f}")
print(f"Test Score: {test_score:.4f}")
print(f"Difference (Train - Test): {difference:.4f}")

print("Ensemble Confusion Matrix\n", conf_matrix_ensemble)
print("\nEnsemble Classification Report:\n", class_report_ensemble)
print("\nEnsemble ROC-AUC:", roc_auc_ensemble)

Train Score: 0.8078
Test Score: 0.7899
Difference (Train - Test): 0.0179
Ensemble Confusion Matrix
 [[852 183]
 [113 261]]

Ensemble Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.82      0.85      1035
           1       0.59      0.70      0.64       374

    accuracy                           0.79      1409
   macro avg       0.74      0.76      0.75      1409
weighted avg       0.80      0.79      0.80      1409


Ensemble ROC-AUC: 0.850727221059702


We selected the ensemble_model as the most balanced predictive model for our case.

In [114]:
log_transform_columns = ['TotalCharges', 'MonthlyCharges']

In [115]:
# Preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        # Apply the logarithmic transformation
        ('log', FunctionTransformer(np.log1p, validate=True), log_transform_columns),
        # Leave the other columns unchanged
        ('rest', 'passthrough', [col for col in X_train.columns if col not in log_transform_columns])
    ]
)

In [116]:
# Models
log_model = LogisticRegression(
    C=0.5908361216819946,
    class_weight='balanced',
    max_iter=100,
    penalty='l2',
    solver='newton-cg',
    random_state=42
)

xgb_model = XGBClassifier(
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss',
    subsample=0.8,
    n_estimators=400,
    min_child_weight=7,
    max_depth=5,
    learning_rate=0.01,
    gamma=0.3,
    colsample_bytree=1.0
)

In [117]:
# Ensamble
ensemble_model = VotingClassifier(
    estimators=[
        ('logistic', log_model),
        ('xgboost', xgb_model)
    ],
    voting='soft'
)

In [118]:
# Pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),          
    ('scaler', StandardScaler()),            
    ('ensemble', ensemble_model)             
])

In [119]:
pipeline.fit(X_train, y_train)

/Users/laraaldalur/Desktop/thebridge/2409_dsft_lara_core/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:27:48] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('log',
                                                  FunctionTransformer(func=<ufunc 'log1p'>,
                                                                      validate=True),
                                                  ['TotalCharges',
                                                   'MonthlyCharges']),
                                                 ('rest', 'passthrough',
                                                  ['SeniorCitizen', 'Partner',
                                                   'Dependents', 'tenure',
                                                   'PhoneService',
                                                   'PaperlessBilling',
                                                   'gender_Male',
                                                   'MultipleLines_Yes',
                                                   'InternetService_Fiber '
                                                   'optic',
                                                   'InternetService_No...
                                                             grow_policy=None,
                                                             importance_type=None,
                                                             interaction_constraints=None,
                                                             learning_rate=0.01,
                                                             max_bin=None,
                                                             max_cat_threshold=None,
                                                             max_cat_to_onehot=None,
                                                             max_delta_step=None,
                                                             max_depth=5,
                                                             max_leaves=None,
                                                             min_child_weight=7,
                                                             missing=nan,
                                                             monotone_constraints=None,
                                                             multi_strategy=None,
                                                             n_estimators=400,
                                                             n_jobs=None,
                                                             num_parallel_tree=None,
                                                             random_state=42, ...))],
                                  voting='soft'))])

In [120]:
y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
y_pred = (y_pred_proba >= 0.5).astype(int)

conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print("Confusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", class_report)
print("\nROC-AUC:", roc_auc)

Confusion Matrix:
 [[852 183]
 [113 261]]

Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.82      0.85      1035
           1       0.59      0.70      0.64       374

    accuracy                           0.79      1409
   macro avg       0.74      0.76      0.75      1409
weighted avg       0.80      0.79      0.80      1409


ROC-AUC: 0.8507246376811595


#### Threshold try on test

In [121]:
# We test with the recommended threshold

y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
y_pred = (y_pred_proba >= 0.47).astype(int)


conf_matrix = confusion_matrix(y_test, y_pred_adjusted)
class_report = classification_report(y_test, y_pred_adjusted)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print("Adjusted Threshold Confusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", class_report)
print("\nROC-AUC:", roc_auc)

Adjusted Threshold Confusion Matrix:
 [[831 204]
 [101 273]]

Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.80      0.84      1035
           1       0.57      0.73      0.64       374

    accuracy                           0.78      1409
   macro avg       0.73      0.77      0.74      1409
weighted avg       0.81      0.78      0.79      1409


ROC-AUC: 0.8507246376811595


In [123]:
import joblib

# Save pipeline
joblib.dump(pipeline, 'churn_detection_pipeline.pkl')

['churn_detection_pipeline.pkl']

In [124]:
pipeline = joblib.load('churn_detection_pipeline.pkl')
X_full = df_num.drop(columns=['Churn'])  # Features
y_true = df_num['Churn']  


#### Results all data

In [125]:
y_pred_proba = pipeline.predict_proba(X_full)[:, 1]  
y_pred = (y_pred_proba >= 0.5).astype(int)  

conf_matrix = confusion_matrix(y_true, y_pred)
class_report = classification_report(y_true, y_pred)
roc_auc = roc_auc_score(y_true, y_pred_proba)

print("Confusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", class_report)
print("\nROC-AUC:", roc_auc)

Matriz de Confusión:
 [[4349  825]
 [ 554 1315]]

Informe de Clasificación:
               precision    recall  f1-score   support

           0       0.89      0.84      0.86      5174
           1       0.61      0.70      0.66      1869

    accuracy                           0.80      7043
   macro avg       0.75      0.77      0.76      7043
weighted avg       0.81      0.80      0.81      7043


ROC-AUC: 0.8655741149671476


#### Threshold try on all data

In [126]:
# We test with the recommended threshold

y_pred_proba = pipeline.predict_proba(X_full)[:, 1]  
y_pred = (y_pred_proba >= 0.47).astype(int)  

conf_matrix = confusion_matrix(y_true, y_pred)
class_report = classification_report(y_true, y_pred)
roc_auc = roc_auc_score(y_true, y_pred_proba)

print("Adjusted Threshold Confusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", class_report)
print("\nROC-AUC:", roc_auc)

Adjusted Threshold Confusion Matrix:
 [[4234  940]
 [ 493 1376]]

Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.82      0.86      5174
           1       0.59      0.74      0.66      1869

    accuracy                           0.80      7043
   macro avg       0.74      0.78      0.76      7043
weighted avg       0.82      0.80      0.80      7043


ROC-AUC: 0.8655741149671476
